In [227]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def MSE(x, y):
    return np.sum((x - y)**2) / len(x)

def lin_act(x):
    return x

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def grad_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

class Layer:   
    def __init__(self, neurons, input_shape, weights, bias, activation):
        self.neurons = neurons
        self.input_shape = input_shape
        
        assert weights.shape == (input_shape[1], neurons)
        self.weights = weights
        
        assert bias.shape == (1, neurons)
        self.bias = bias
        
        self.activation = activation
        self.last_a = None
    
    
    def make_factory(neurons, input_shape, activation, factory):
        return Layer(
            neurons = neurons,
            input_shape = input_shape,
            weights = factory((input_shape[1], neurons)),
            bias = factory((1, neurons)),
            activation = activation
        )
    
    def make_zero(neurons, input_shape, activation):
        return Layer.make_factory(neurons, input_shape, activation, np.zeros)
    
    def make_random(neurons, input_shape, activation):
        random_balanced = lambda shape: np.random.random(shape)
        return Layer.make_factory(neurons, input_shape, activation, random_balanced)
    
    def apply(self, inputs):
        intensities = inputs @ self.weights
        self.last_a = intensities + self.bias
        return self.activation(intensities + self.bias)
    
    def apply_no_intensities(self, inputs):
        intensities = inputs @ self.weights
        return self.activation(intensities + self.bias)
    
    def __str__(self):
        return f"LAYER(\nW:\n {repr(self.weights)} \nb:\n{repr(self.bias)})\n"
    
    def __repr__(self):
        return str(self)


class NN:
    def __init__(self, *layers, input_shape):
        self.input_shape = input_shape
        self.layers = [*layers]
        self.errors = None
        self.last_inputs = None
        
    def get_last_shape(self):
        if self.layers:
            return self.layers[-1].weights.shape
        else:
            return self.input_shape
    
    def add_new_zero_layer(self, neurons, activation=sigmoid):
        layer = Layer.make_zero(
            neurons,
            self.get_last_shape(),
            activation
        )
        self.layers.append(layer)
        return layer
    
    def add_new_random_layer(self, neurons, activation=sigmoid):
        layer = Layer.make_random(
            neurons,
            self.get_last_shape(),
            activation
        )
        self.layers.append(layer)
        return layer
        
    def apply(self, inputs):
        self.last_inputs = inputs
        x = inputs
        for layer in self.layers:
            x = layer.apply(x)
        return x
    
    def backpropagate(self, yhat, y, batch_size):
        # this shit dont work
        errors = [None] * len(self.layers)
        errors[-1] = grad_sigmoid(self.layers[-1].last_a) *  (yhat - y)
        for i in range(2, len(errors) + 1):
            uhm = errors[-i+1] @ np.transpose(self.layers[-i+1].weights)
            errors[-i] = grad_sigmoid(self.layers[-i].last_a) * uhm
        print(errors)
        grad = [None] * len(self.layers)
        grad_b = [None] * len(self.layers)
        for k in range(len(errors)):
            if k == 0:
                f_a = self.layers[0].activation(self.last_inputs)
            else:
                cur_layer = self.layers[k-1]
                f_a = cur_layer.activation(cur_layer.last_a)
            
            grad[k] = np.transpose(f_a) @ errors[k]
            grad_b[k] = np.ones((1,batch_size)) @ errors[k] 
                
        for k, layer in enumerate(self.layers):
            #print(grad[k].shape)
            layer.weights -= 1e-3 * grad[k]
            layer.bias -= 1e-3 * grad_b[k]
        return grad
    
class Trainer:
    def __init__(self, nn):
        self.nn = nn
        
    def backpropagate(self, x, y):
        yhat = nn.apply(x)
        

In [233]:
nn = NN(input_shape=(_, 1))
hl1 = nn.add_new_random_layer(5)
#hl2 = nn.add_new_random_layer(5)
ol = nn.add_new_random_layer(1, activation=lin_act)
ol.bias = np.zeros((1,1))

In [234]:
nn.apply(np.array([[1]]))

array([[1.19973346]])

In [235]:
nn.layers

[LAYER(
 W:
  array([[0.88859829, 0.98225591, 0.12419479, 0.64327458, 0.88579401]]) 
 b:
 array([[0.35062152, 0.48505979, 0.72049584, 0.32404472, 0.36443597]])),
 LAYER(
 W:
  array([[0.21580875],
        [0.22740014],
        [0.09739063],
        [0.21369294],
        [0.80355348]]) 
 b:
 array([[0.]]))]

In [236]:
for i in range(3):
    out = nn.apply(np.array([[1], [2], [3]]))
    nn.backpropagate(out, np.array([[1], [4], [9]]), 3)


[array([[ 0.00133549,  0.00123034,  0.00072755,  0.00151545,  0.00494247],
       [-0.0087243 , -0.00707177, -0.00825629, -0.01260723, -0.03227516],
       [-0.01108937, -0.00796369, -0.02113974, -0.02121346, -0.04108898]]), array([[ 0.03553656],
       [-0.42510541],
       [-1.15446926]])]
[array([[ 0.00138531,  0.00127596,  0.00075848,  0.00157149,  0.00510148],
       [-0.00871855, -0.00706562, -0.00829232, -0.01259484, -0.03209252],
       [-0.01109708, -0.00796759, -0.02126085, -0.021221  , -0.0409094 ]]), array([[ 0.03661634],
       [-0.42200202],
       [-1.1476193 ]])]
[array([[ 0.00143497,  0.00132142,  0.00078951,  0.00162734,  0.00525865],
       [-0.00871224, -0.00705904, -0.00832691, -0.01258172, -0.03191125],
       [-0.01110403, -0.00797098, -0.02137847, -0.02122721, -0.04073092]]), array([[ 0.03767964],
       [-0.41893187],
       [-1.14083497]])]


In [218]:
sigmoid(1)

0.7310585786300049